In [2]:
import numpy as np 
import pandas as pd 
import os

In [3]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import librosa
import soundfile
import os,glob,pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [140]:
def extract_feature(file_name, mfcc, chroma, mel):
    X, sample_rate = librosa.load(file_name)
    result = np.array([])
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13), axis=1)
        result = np.hstack((result, mfccs))
    if chroma:
        chroma = np.mean(librosa.feature.chroma_stft(y=X, sr=sample_rate), axis=1)
        result = np.hstack((result, chroma))
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate), axis=1)
        result = np.hstack((result, mel))
    return result

In [141]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [142]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/kaggle/input/ravdess-emotional-speech-audio/Actor*/*"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [143]:
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [144]:
x_train

array([[-6.79428040e+02,  7.15642624e+01, -1.25025034e+00, ...,
         8.96288293e-06,  3.88968374e-06,  2.31587521e-07],
       [-7.42097412e+02,  5.40084419e+01,  1.90167465e+01, ...,
         1.17418485e-05,  2.54501492e-06,  1.63714375e-07],
       [-5.99767761e+02,  3.90904655e+01, -1.20699177e+01, ...,
         3.52484116e-04,  2.07809018e-04,  9.77575201e-06],
       ...,
       [-5.16340942e+02,  2.82089615e+01, -1.60243416e+01, ...,
         1.25860868e-04,  7.58955066e-05,  8.02706745e-06],
       [-7.14336975e+02,  4.53424454e+01, -3.30322075e+00, ...,
         9.63522598e-06,  5.22228902e-06,  3.50979036e-07],
       [-6.11058960e+02,  6.97433624e+01, -1.14987011e+01, ...,
         4.15070936e-05,  3.99114688e-05,  2.02275328e-06]])

In [145]:
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [146]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 153


In [147]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [148]:
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [149]:
y_pred=model.predict(x_test)

In [150]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

from sklearn.metrics import accuracy_score, f1_score

Accuracy: 58.85%


In [151]:
from sklearn.ensemble import RandomForestClassifier

In [152]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = load_data(test_size=0.25)

# Train the random forest classifier on the selected features
rf_selected = RandomForestClassifier()
rf_selected.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = rf_selected.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6666666666666666


In [153]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
bagging = BaggingClassifier(KNeighborsClassifier(),
                            max_samples=0.5, max_features=0.5)
bagging.fit(X_train, y_train)
Y_pred=bagging.predict(X_test)
print("Accuracy of Bagging classifier:",accuracy_score(y_test,Y_pred))

Accuracy of Bagging classifier: 0.5416666666666666


In [154]:
from sklearn.tree import DecisionTreeClassifier
classfier=DecisionTreeClassifier(criterion="entropy",random_state=42)
classfier.fit(X_train,y_train)
Y_pred=classfier.predict(X_test)
print("Accuracy of Decision tree:",accuracy_score(y_test,Y_pred))

Accuracy of Decision tree: 0.5260416666666666


In [155]:
from sklearn.linear_model import LogisticRegression
re=LogisticRegression()
history=re.fit(X_train,y_train)
Y_pred=re.predict(X_test)
print("Accuracy of Logistic Regression:",accuracy_score(y_test,Y_pred))

Accuracy of Logistic Regression: 0.546875


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [156]:

from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(n_estimators=100)
ada.fit(X_train,y_train)
yp=ada.predict(X_test)
print("Accuracy of AdaBoost classifier:",accuracy_score(y_test,yp))

Accuracy of AdaBoost classifier: 0.5


In [157]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train,y_train)
gb=gnb.predict(X_test)
print("Accuracy of GaussianNB:",accuracy_score(y_test,gb))

Accuracy of GaussianNB: 0.4739583333333333


In [158]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
SV = make_pipeline( SVC(gamma='auto'))
SV.fit(X_train,y_train)
SVC=SV.predict(X_test)

print("Accuracy of support vector classifier:",accuracy_score(y_test,SVC))

Accuracy of support vector classifier: 0.59375


In [159]:
f1_score(y_test, y_pred,average=None)

array([0.77083333, 0.61538462, 0.69306931, 0.57831325])

## Thank You